most of this is a random classification on IRIS data. Also the MlFlow Experiment and Runs, incl. Registration of Models can be handled differently, but complete the target of a e2e ml project in databricks /aml

In [ ]:
%pip install mlflow
!pip install sklearn
!pip instal pandas
!pip install azureml-core
%pip install onnxmltools
%pip install onnxruntime
%pip install skl2onnx


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import mlflow.pyfunc
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import cloudpickle
import time
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import get_latest_tested_opset_version
from onnxmltools.utils import save_model


In [ ]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [ ]:
class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
  def __init__(self, model):
    self.model = model

  def predict(self, context, model_input):
    return self.model.predict_proba(model_input)[:,1]

with mlflow.start_run(run_name='edge_run') as run:
    #n_estimators = 10
    model = RandomForestClassifier(random_state=np.random.RandomState(188453))
    model.fit(X_train, y_train)

    target_opset = get_latest_tested_opset_version()
    n_features = X_train.shape[1]
    onnx_clf = convert_sklearn(
        model,
        "edge_job",
        initial_types=[("input", FloatTensorType([None, n_features]))],
        target_opset={"": target_opset, "ai.onnx.ml": 1}
    )
    save_model(onnx_clf, "model.onnx")

    predictions_test = model.predict_proba(X_test)[:,1]

    wrappedModel = SklearnModelWrapper(model)

    signature = infer_signature(X_train, wrappedModel.predict(None, X_train))

    conda_env =  _mlflow_conda_env(
        additional_conda_deps=None,
        additional_pip_deps=["cloudpickle=={}".format(cloudpickle.__version__), "scikit-learn=={}".format(sklearn.__version__)],
        additional_conda_channels=None,
    )
    run_id = run.info.run_id

    mlflow.pyfunc.log_model("edge_model", python_model=wrappedModel, conda_env=conda_env, signature=signature)
    mlflow.end_run()


In [ ]:
mlflow.search_runs(filter_string=f'tags.mlflow.runName = "edge_run" and tags.mlflow.rootRunId = "{run_id}"')


In [ ]:
model_name = "edge_iris"
model_version = mlflow.register_model(f"runs:/{run_id}/edge_model", model_name)

time.sleep(15)


from here training the ML is done, we just prepare the tun script for the later inference.

In [ ]:
import pickle

def inference(inp):

    loaded_model = pickle.load(open("python_model.pkl", 'rb'))
    print(loaded_model.predict(context=None, model_input=[inp]))

if __name__ == '__main__':
    inference((inp))
